In [20]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import sqlite3
import webbrowser

def insertDB(df):
    # 데이터베이스 최신화
    con = sqlite3.connect('./db/news.db')
    cur = con.cursor()
    for i in df.itertuples():
        q = "INSERT into news(titles, url, type, editor, dates) VALUES (?,?,?,?,?)" 
        cur.execute(q, (i[1], i[2], i[3], i[4], i[5]))
    con.commit()
    con.close()

def KLN():
    global KLN_df
    # KLN 물류신문

    # html page 요청 (불러오기)
    page = requests.get(f'https://www.klnews.co.kr/news/articleList.html?page={pageVar+1}&total=67684&box_idxno=&view_type=sm')
    soup = bs(page.text, "html.parser")

    # 필요한 정보 획득
    titles = soup.find_all('h4', 'titles')
    dates = soup.find_all('span', 'byline')

    titles_col = []
    url_col = []
    type_col = []
    editor_col = []
    dates_col = []

    for i, j in zip(titles[:-1], dates[:-1]):
        titles_col.append(i.get_text())
        url_col.append("https://www.klnews.co.kr"+i.a['href'])          
        info = j.find_all('em')
        type_col.append(info[0].get_text())
        editor_col.append(info[1].get_text())
        dates_col.append(info[2].get_text())

    # 데이터프레임화
    KLN_df = pd.DataFrame()
    KLN_df['titles'] = titles_col
    KLN_df['url'] = url_col
    KLN_df['type'] = type_col
    KLN_df['editor'] = editor_col
    KLN_df['dates'] = dates_col
    
#     KLN_df['dates'] = pd.to_datetime(KLN_df['dates'])
#     KLN_df['year'] = [i.year for i in KLN_df['dates']]
#     KLN_df['month'] = [i.month for i in KLN_df['dates']]
#     KLN_df['day'] = [i.day for i in KLN_df['dates']]
    
    insertDB(KLN_df)
    
def removeDuplicates():
    pass
#     # 데이터베이스 중복제거
#     con = sqlite3.connect('./db/news.db')
#     cur = con.cursor()
#     q = 'DELETE FROM news where titles in (SELECT titles FROM news GROUP BY titles, dates HAVING count(*) > 1)'
#     cur.execute(q)
#     con.commit()
#     con.close()
    
def loadDB(pageVar):
    global df_news
    con = sqlite3.connect('./db/news.db')
    cur = con.cursor()
    q = f"SELECT * FROM news LIMIT {pageVar * numOfcontents}, {numOfcontents}"
    tmp = pd.read_sql_query(q, con)
    con.close()
    return tmp
    

In [21]:
import tkinter as tk

win = tk.Tk()

win.title('물류 늬우스')
win.geometry("640x400+100+100")
win.resizable(False, False)

pageVar = 0
numOfcontents = 20

frame_menu = tk.Frame(win, relief="solid", bd=2)
frame_menu.place(x=100, y=110)

frame_control_listbox = tk.Frame(win, relief='solid', bd=2)
frame_control_listbox.place(x=500, y=110)

frame_Top = tk.Frame(win)
frame_Top.place(x=0,y=0)


frame_listbox = tk.Frame(win, relief="solid", bd=2)
scrollbar=tk.Scrollbar(frame_listbox)
scrollbar.pack(side="right", fill="y")

listbox = tk.Listbox(frame_listbox, selectmode='browse', height=10, width=50, yscrollcommand = scrollbar.set)
listbox.pack()

scrollbar["command"]=listbox.yview

frame_listbox.place(x=100, y=150)

logo = tk.PhotoImage(file="./img/logo2.png")
lbl_logo = tk.Label(frame_Top, image=logo)
lbl_logo.pack()

def pageShow():
    global df_news
    df_news = loadDB(pageVar)
    for i in range(numOfcontents):
        listbox.insert(i, f"{pageVar * 10 + (i+1)}번, {df_news['titles'][pageVar * 10 + i]}")
    listbox.pack()
    
def clearListbox():
    listbox.delete(0,numOfcontents-1)

def nextPage():
    global pageVar
    clearListbox()
    pageVar += 1
#     if pageVar % 2 != 0:
#         update()
    pageShow()
   
def prevPage():
    global pageVar
    clearListbox()
    if pageVar > 0:
        pageVar -= 1
    pageShow()

def update():
    KLN()

def updateNews():
    global pageVar
    pageVar = 0
    
    update()
    clearListbox()
    pageShow()

def applyCondition():
    print(etr_keywordEntry.get()) # 키워드가 포함된 기사 추출
    
def clear(event):
    etr_keywordEntry.delete(0, len(etr_keywordEntry.get()))

def openUrl():
    if listbox.curselection() == ():
        print(None) ## 에러메시지 : 기사를 선택해주세요.
    else:
        idx = listbox.curselection()
        url = df_news.iloc[idx]['url']
        webbrowser.open(url)

    


btn_updateNews = tk.Button(frame_menu, text='최신화', command=updateNews)
btn_updateNews.pack(side='left', padx=2, pady=1)

btn_applyCondition = tk.Button(frame_menu, text='검색', command=applyCondition)
btn_applyCondition.pack(side='left', padx=2, pady=1)

etr_keywordEntry = tk.Entry(frame_menu, width=36, bg='light blue')
etr_keywordEntry.insert(0,"키워드를 입력해주세요.")
etr_keywordEntry.bind("<Button-1>", clear)
etr_keywordEntry.pack(side='left', padx=5, pady=1)

btn_prevPage = tk.Button(frame_control_listbox, text='<<', command=prevPage)
btn_prevPage.pack()
    
btn_nextPage = tk.Button(frame_control_listbox, text='>>', command=nextPage)
btn_nextPage.pack()

btn_openUrl = tk.Button(frame_control_listbox, text='열기', command=openUrl)
btn_openUrl.pack()


win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\12182\anaconda3\lib\site-packages\pandas\core\indexes\range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 20 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\12182\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\12182\AppData\Local\Temp/ipykernel_22788/1190049200.py", line 53, in nextPage
    pageShow()
  File "C:\Users\12182\AppData\Local\Temp/ipykernel_22788/1190049200.py", line 41, in pageShow
    listbox.insert(i, f"{pageVar * 10 + (i+1)}번, {df_news['titles'][pageVar * 10 + i]}")
  File "C:\Users\12182\anaconda3\lib\site-packages\pandas\core\series.py", line 958, in __getitem__
    return self._get_value(key)
  File "C:\Users\12182\anaconda3\lib\site-packages\pandas\core\series.py", line 1069, in _get_value
    loc = self.index.